# Two-Dimensional Trajectories
## Lecture 5

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import interactive

We now turn our attention to two-dimensional projectile motion.  Our specific example will be the shell shot by a large cannon.

In [ ]:
from IPython.display import YouTubeVideo

YouTubeVideo("hlW6hZkgmkA", playlist="hlW6hZkgmkA", autoplay=1, loop=1)

In the absence of air-resistance, the equations of motion (Newton's 2nd Law) are:

$$
\begin{eqnarray}
m\frac{d^2 x}{dt^2} =& 0 \\
m\frac{d^2 y}{dt^2} =& -mg
\end{eqnarray}
$$

These are second-order differential equations.  A standard technique for solving higher order differential equations is to rewrite them as a system of first order differential equations.  That is,

$$
\begin{eqnarray}
\frac{d x}{dt} =& v_x \\
m\frac{d v_x}{dt} =& 0 \\
\frac{d y}{dt} =& v_y \\
m\frac{d v_y}{dt} =& -m g
\end{eqnarray}
$$

There are now four equations written out for the variables $x, y, v_x, v_y$.  As before, we can solve this problem by using Euler's method:

$$
\begin{align}
x_{i+1} &= x_i + v_{x,i} \Delta t \\
v_{x,i+1} &= v_{x,i} \\
y_{i+1} &= y_i + v_{y,i} \Delta t \\
v_{y,i+1} &= v_{y,i} - g \Delta t
\end{align}
$$

This algorithm allows us to estimate the position and velocity of a projectile given the intial values of $x, y, v_x$, and $v_y$. In this notation the subscript $_{i+1}$ indicate the next time step and the subscript $_{i}$ indicates the current time step. 

This notation is used in the sense

$$
\begin{align}
x_i &= x(t_i) \\
x_{i+1} &= x(t_{i+1i}) \\
v_{y, i+1} &= v_y(t_{i+i})
\end{align}
$$

and so on for each variable. The idea is we are evaluating the function $x(t)$, $y(t)$, $v_x(t)$, and $v_y(t)$ and the specific points $t = t_0, t_1, t_2, \ldots, t_i, t_{i+1}, \ldots$


Like in the falling particle problem, air resistance is important. To introduce air resistance we can model the drag as

$$ F_{d} = - C_2 v^2 $$

like we did in the falling particle problem where $v$ is the magnitude of the velocity vector. Since force is vector so we need to split this into horizontal and vertical components.

This force is a vector and always directed in the opposite direction of the velocity vector.

\begin{align}
F_{d,x} = F_{d} \cos \theta = F_{d} (v_x / v) \\
F_{d,y} = F_{d} \sin \theta = F_{d} (v_y / v)
\end{align}

we can write

$$
\begin{align}
F_{d, x} = - C_2 v v_x \\
F_{d, y} = - C_2 v v_y 
\end{align}
$$

*Whiteboard exercise to draw out these vectors.*

So the new equations of motion will be

$$
\begin{eqnarray}
\frac{d x}{dt} =& v_x \\
m \frac{d v_x}{dt} =& F_{d,x} \\
\frac{d y}{dt} =& v_y \\
m\frac{d v_y}{dt} =& -mg + F_{d,y}
\end{eqnarray}
$$

- - - 
#### Euler's Method

To numerically solve the differential equation

$$\begin{align}
\frac{d y}{dt} &= f(y, t) \\
y(t_0) &= y_0
\end{align}$$

we can use the Euler's method algorithm

$$\begin{align}
y(t_0) &= y_0 \\
y(t_{i+1}) &= y(t_i) + f(y(t_i), t_i) \Delta t
\end{align}
$$
- - -

We can add these drag forces to our algorithm like we did last previously.


\begin{align}
x_{i+1} &= x_i + v_{x,i} \Delta t \\
v_{x,i+1} &= v_{x,i} + \left(- \frac{C_2 v_i v_{x,i}}{m}\right) \Delta t \\
y_{i+1} &= y_i + v_{y,i} \Delta t \\
v_{y,i+1} &= v_{y,i} + \left(- g  - \frac{C_2 v_i v_{y,i}}{m}\right) \Delta t
\end{align}

Here the term $v_i$ means the speed at time step $t_i$ and where the speed is calculated as $v = \sqrt{v_x^2 + v_y^2}$.

In projectile motion for particle without air resistance, one of the strategies we used was to treat the vertical and horizontal motion independently. We can that the horizontal and vertical motion are *decoupled*.

In the situation with air resistance the horizontal and vertical motion are *coupled* together through the speed term. They must be solved for simultaneously.

### Simulation of position vs. time for a cannon ball

Let's start with the same prototype code we had the beginning of Lecture 4.  Using previously written code as a template, framework, skeleton, or "cookie-cutter" is a great way to make a quick start on a new problem. 

In [ ]:
def FallingParticleApp():
    
    # set up the model
    model = FallingParticleStepper()
    
    # iterate the model
    ball = pd.DataFrame(model)
    
    # plot the results
    FallingParticlePlot(ball)

We separate out the code that will be iterated to step through time to actual solve the numerical program:

In [ ]:
g = 9.8 # define a global constant

def FallingParticleStepper(dt=0.01, y0=100, v0=0, tmax=5):
    
    # initialize the model
    t = 0
    y = y0
    v = v0
    
    while True:
        
        model = {'t': t, 'y': y, 'v': v}
        yield model # return the model state back to the caller
            
        if t > tmax:
            break
            
        # use the Euler algorithm to update the state of the model
        y = y + v*dt
        v = v - g*dt
        t = t + dt

And the routines to visualize and plot the results:

In [ ]:
def FallingParticlePlot(particle):
    
    plt.plot(particle.t, particle.y)
    plt.ylabel('y (m)')
    plt.xlabel('t (s)')

In [ ]:
FallingParticleApp()

So now we can follow the pattern. First thing to write is a driver program:

In [ ]:
def ProjectileMotionApp():
    
    # set up the model
    model = ProjectileMotionStepper()
    
    # iterate the model
    ball = pd.DataFrame(model)
    
    # plot the results
    ProjectileMotionPlot(ball)

The only changes were to replace the phase `FallingParticle` with `ProjectileMotion`.  

Why start with the driver program? Because this allows use to practice *test-driven development*.  Right away we can run our main driver program and see the results or discover any remaining issues we need to fix:

In [ ]:
ProjectileMotionApp()

This `NameError` just means that ProjectileMotionStepper hasn't been defined yet. But as soon at this and the related plotting function are written we have a test program we can run right away.

Next with computational model itself, `ProjectileMotionStepper()`

In this problem, the model variables are now `x`, `y`, `vx`, and `vy`.  Let's add initial conditions for each of those variables, add them to the model, and code up the new lines needed in the Euler's method section.

In [ ]:
def ProjectileMotionStepper(dt=0.01, 
                            x0=0, y0=0, 
                            vx0=30, vy0=30,
                            tmax=6, m=1):
    
    # initialize the model
    t = 0
    x = x0
    y = y0
    vx = vx0
    vy = vy0
    
    while True:
        
        model = {'t': t, 
                 'x': x, 'y': y,
                 'vx': vx, 'vy': vy}
        yield model # return the model state back to the caller
    
        if t > tmax:
            break
        
        # we will need to add in drag eventually
        # TODO
        Fdx = 0
        Fdy = 0
        
        # use the Euler algorithm to update the state of the model
        x = x + vx*dt
        y = y + vy*dt
        vx = vx + Fdx/m * dt
        vy = vy + (Fdy/m - g) * dt
        t = t + dt

Notice that we didn't bother filling in the `Fdx` and `Fdy` lines yet.  They require a little bit of thinking so we stub the codes and leave a note to ourselves `# TODO` to remind us to get back to this part of the code.

Finally, a basic plotting function `ProjectileMotionPlot()`. There are lots of plots we could make the simplest may be just plotting the vertical coordinate $y$ against the horizontal coordinate $x$.

In [ ]:
def ProjectileMotionPlot(particle):
    
    plt.plot(particle.x, particle.y)
    plt.ylabel('y (m)')
    plt.xlabel('x (m)')

Now that we have the basic pieces, time to test our code:

In [ ]:
ProjectileMotionApp()

*Exercise* There is `NameError` that we should fix now. 

*Discuss* How to read Python error messages.

Now, with the errors resolved, we can tackle whether the results look resonable.  What is the plot showing? What do we expect it to look like?

*Exercise* Modify the default initial conditions in the function definition of `ProjectileMotionStepper()` so that we get a classical parabolic trajectory.

In [ ]:
ProjectileMotionApp()

To confirm that our numerical model is working working correctly, it can be helpful to compare with a known analytical results.  For the case of no air resistance, we know:

$$\begin{align}
y &= y_0 + v_{y,0} t- \frac{1}{2} g t^2 \\
x &= x_0 + v_{x,0} t 
\end{align}$$

or eliminating $t$, 

$$y = \frac{v_{y,0}}{v_{x, 0}} (x-x_0) − \frac{1}{2}\frac{g}{ v_{x,0}^2}(x-x_0)^2$$

We can improve our visualization if we add this information to the plotting function.

In [ ]:
model = ProjectileMotionStepper()
ball = pd.DataFrame(model)
ball.iloc[0]

In [ ]:
def ProjectileMotionPlot(particle, style='', legend=True):
    # style is optional parameter to control color, linestyle, and marker used
    
    plt.plot(particle.x, particle.y, style, label='Numerical')
    plt.ylabel('y (m)')
    plt.xlabel('x (m)')
    
    # extract out initial conditions
    init = particle.iloc[0]
    x0 = init.x
    y0 = init.y
    vx0 = init.vx
    vy0 = init.vy
   
    # Analytical solution
    x = particle.x
    y = y0 + vy0/vx0 * (x-x0) - 1/2 * g/vx0**2 * (x-x0)**2
    plt.plot(x, y, style, label='Analytical')
    
    # draw the ground level, y=0
    plt.axhline(0, color='k')
    
    # ensure aspect ratio is square
    plt.axis('equal')
    if legend:
        plt.legend()

In [ ]:
ProjectileMotionApp()

Now let add in the air drag.

Let's reproduce our `ProjectileMotionStepper` code again here with the changes we have made. Now it is time finish off implementing the drag forces. We will also need to add a new parameter, `C2`, to control the air drag.

In [ ]:
def ProjectileMotionStepper(dt=0.01,
                            m=1, C2=0.005,
                            x0=0, y0=0, 
                            vx0=30, vy0=30,
                            tmax=6):
    
    # initialize the model
    t = 0
    x = x0
    y = y0
    vx = vx0
    vy = vy0
    
    while True:
        
        model = {'t': t, 
                 'x': x, 'y': y,
                 'vx': vx, 'vy': vy}
        yield model # return the model state back to the caller
    
        if t > tmax:
            break
        if y < 0: # stop if the projectile hits the ground
            break
            
        # drag forces
        v = np.sqrt(vx**2+vy**2)
        Fdx = -C2*v*vx
        Fdy = -C2*v*vy
        
        # use the Euler algorithm to update the state of the model
        x = x + vx*dt
        y = y + vy*dt
        vx = vx + Fdx/m * dt
        vy = vy + (Fdy/m - g) * dt
        t = t + dt

In [ ]:
ProjectileMotionApp()

There are now a number of parameters we may want to vary. Rather than trying to write one master driver program that tries to do everything it can be easier to have programs for specific purposes.

In [ ]:
def VaryDragCoefficientApp(C2=0):
    
    # set up the model
    model = ProjectileMotionStepper(C2=C2)
    
    # iterate the model
    ball = pd.DataFrame(model)
    
    # plot the results
    ProjectileMotionPlot(ball)
    plt.xlim(0, 200)
    
interactive(VaryDragCoefficientApp, C2=(0, 0.02, 0.001))

- - -

### **Problem 3.10** Trajectory of a steel ball

a. Compute the two-dimensional trajectory of a ball moving in air without air friction, and plot $y$ as a function of $x$. 

Compare your computed results with the exact results. For example, assume that a ball is thrown from ground level at an angle $\theta_0$ above the horizontal with an initial velocity of $v_0= 15\;\mbox{m/s}$. 

Vary $\theta_0$ and show that the maximum range occurs at $\theta_0=\theta_{max}=45^\circ$. What is $R_{max}$, the maximum range, at this angle? Compare your numerical result to the analytical result $R_{max}=v_0^2/g$.

In [ ]:
def Problem310aApp(angle=45):
    
    v0 = 15
    θ0 = np.deg2rad(angle)
    vx0 = v0*np.cos(θ0)
    vy0 = v0*np.sin(θ0)
    
    print("vx0 = {:.2f} vy0 = {:.2f}".format(vx0, vy0))

    # set up the model
    model = ProjectileMotionStepper(C2=0,
                                    vx0=vx0, vy0=vy0,
                                    x0=0, y0=0)
    
    # iterate the model
    ball = pd.DataFrame(model)
    
    # plot the results
    ProjectileMotionPlot(ball, style='-')
    plt.xlim(0, 30)

    # determine the range
    # use max to determine the range?
    #R = max(ball.x)
    # use average of last points ?
    #R = (ball.iloc[-1].x + ball.iloc[-2].x)/2
    # linear interpolation y-y1 = (y2-y1)/(x2-x1)*(x-x1)
    x1 = ball.iloc[-2].x
    x2 = ball.iloc[-1].x
    y1 = ball.iloc[-2].y
    y2 = ball.iloc[-1].y
    # solve for x: x-x1 = (x2-x1)/(y2-y1)*(y-y1)
    # evaluate at y = 0:
    R = x1 + (x2-x1)/(y2-y1)*(0-y1)
    
    Rmax = v0**2 / g
    plt.axvline(Rmax, color='r')
    plt.axvline(R)
    
    print("R = {:.2f} Rmax = {:.2f}".format(R, Rmax))
  
    #plt.axis('auto')
    
    #plt.xlim(22, 24)
    #plt.ylim(-0.5, 0.5)
    
interactive(Problem310aApp, angle=(0, 90, 5))

b. Suppose that a steel ball is thrown from a height $h$ at an angle $\theta_0$ above the horizontal with the same initial speed as in part (a). If you neglect air resistance, do you expect $\theta_{max}$ to be larger or smaller than $45^\circ$? What is $\theta_{max}$ for $h= 2\;\mbox{m}$? By what percent is the range $R$ changed if $\theta$ is varied by 2% from $\theta_{max}$?

In [ ]:
def Problem310bApp(h=2):
   
    v0 = 15
    angles = np.arange(0, 90, 5)
    ranges = []
    
    for angle in angles:
        θ0 = np.deg2rad(angle)

        vx0 = v0 * np.cos(θ0)
        vy0 = v0 * np.sin(θ0)

        # set up the model
        model = ProjectileMotionStepper(dt=0.01, 
                                        y0=h, 
                                        vx0=vx0, vy0=vy0, 
                                        C2=0.0)

        # iterate the model
        ball = pd.DataFrame(model)

        # determine maximum range
        x1 = ball.x.iloc[-2]
        x2 = ball.x.iloc[-1]
        y1 = ball.y.iloc[-2]
        y2 = ball.y.iloc[-1]
        # solve for x: x-x1 = (x2-x1)/(y2-y1)*(y-y1)
        # evaluate at = 0:
        R = x1 + (x2-x1)/(y2-y1)*(0-y1)
        
        ranges.append(R)
    
    fig, axs = plt.subplots(1, 2, figsize=(10, 4))
    axs[0].plot(angles, ranges, 'o-')
    axs[0].set_xlabel('Angle')
    axs[0].set_ylabel('Range (m)')
    
    # need to figure out angle that gives the max range
    # maximum occurs when derivative is zero, dR/da = 0
    dR = np.diff(ranges)
    da = np.diff(angles)
    dRda = dR / da
    a = (angles[1:]+angles[:-1])/2
    
    axs[1].plot(a, dRda, 'o-')
    axs[1].set_xlabel('Angle')
    axs[1].set_ylabel('dR/da')
    
    # root finding problem
    for i in range(len(ranges)-1):
        y1 = dRda[i]
        y2 = dRda[i+1]
        x1 = a[i]
        x2 = a[i+1]
        if y1 > 0 and y2 <= 0:
            break
    # linear interpolation
    angle_max = x1 + (x2-x1)/(y2-y1)*(0-y1)    
    
    axs[1].axhline(0)
    axs[1].axvline(angle_max)
    print('h = {:.1f} Angle max = {:.2f}'.format(h, angle_max))
        
interactive(Problem310bApp, h=(0, 10, 1))

c. Consider the effects of air resistance on the range and optimum angle of a steel ball. For a ball of mass 7 kg and cross-sectional area 0.01 m$^2$, the parameter $C_2 \approx 0.1$. What are the units of $C_2$? It is convenient to exaggerate the effects of air resistance so that you can more easily determine the qualitative nature of the effects. Hence, compute the optimum angle for $h= 2\;\mbox{m}$, $v_0=30\;\mbox{m/s}$, and $C_2/m=0.1$, and compare your answer to the value found in part (b). Is $R$ more or less sensitive to changes in $\theta_0$ from $\theta_{max}$ than in part (b)? Determine the optimum launch angle and the corresponding range for the more realistic value of $C_2= 0.1$. 

*complete on Assignment 3*

- - -
### Textbook readings

This lecture is based on CSM Chapter 3:

- 3.8 Two-Dimensional Trajectories


